In [1]:
# Import nesessary packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# Load data from pickle
df = pd.read_pickle("Loan_status.pkl")

In [4]:
df.head()

,Unnamed: 0,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag
0,0,1077501,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,735.0,739.0,1.0,NaN,NaN,3.0,0.0,13648.0,83.7%,9.0,f,0.0,0.0,5863.155187,5833.84,5000.00,863.16,0.00,0.0,0.00,Jan-2015,171.62,NaN,May-2020,704.0,700.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
1,1,1077430,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,car,bike,309xx,GA,1.00,0.0,Apr-1999,740.0,744.0,5.0,NaN,NaN,3.0,0.0,1687.0,9.4%,4.0,f,0.0,0.0,1014.530000,1014.53,456.46,435.17,0.00,122.9,1.11,Apr-2013,119.66,NaN,Oct-2016,499.0,0.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2,2,1077175,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,735.0,739.0,2.0,NaN,NaN,2.0,0.0,2956.0,98.5%,10.0,f,0.0,0.0,3

In [5]:
# Read the file 
# df = pd.read_csv("Loan_status_2007-2020Q3.gzip", on_bad_lines="skip", low_memory=False)

In [6]:
# Save file to pickle to save time
# df.to_pickle("Loan_status.pkl")

# Split Dataset into Training, Validation, Testing and Untouched Sets

In [7]:
'''
## Split Dataset
from sklearn.model_selection import train_test_split

# Step 1: Split off untouched data (10%)
df_train_val, df_untouched = train_test_split(df, test_size=0.10, random_state=42)
df_train_val = df_train_val.copy()  # Avoid SettingWithCopyWarning
df_untouched = df_untouched.copy()
df_untouched["set_flag"] = 0  # Mark untouched

# Step 2: Split remaining into train (50%), validation (20%), and test (20%)
df_train, df_temp = train_test_split(df_train_val, test_size=0.40, random_state=42)
df_train = df_train.copy()
df_temp = df_temp.copy()
df_train["set_flag"] = 1  # Mark train

df_val, df_test = train_test_split(df_temp, test_size=0.50, random_state=42)
df_val = df_val.copy()
df_test = df_test.copy()
df_val["set_flag"] = 2  # Mark validation
df_test["set_flag"] = 3  # Mark test

# Combine all subsets back into one dataframe
df_final = pd.concat([df_train, df_val, df_test, df_untouched], ignore_index=True)

# Save as Parquet for efficient storage
df_final.to_parquet("dataset_with_flags.parquet", index=False)

# Check dataset sizes
print(f"Training Set: {len(df_train)} rows ({len(df_train)/len(df)*100:.1f}%)")
print(f"Validation Set: {len(df_val)} rows ({len(df_val)/len(df)*100:.1f}%)")
print(f"Test Set: {len(df_test)} rows ({len(df_test)/len(df)*100:.1f}%)")
print(f"Untouched Set: {len(df_untouched)} rows ({len(df_untouched)/len(df)*100:.1f}%)")

print("Data saved with set_flag column.")
'''

'\n## Split Dataset\nfrom sklearn.model_selection import train_test_split\n\n# Step 1: Split off untouched data (10%)\ndf_train_val, df_untouched = train_test_split(df, test_size=0.10, random_state=42)\ndf_train_val = df_train_val.copy()  # Avoid SettingWithCopyWarning\ndf_untouched = df_untouched.copy()\ndf_untouched["set_flag"] = 0  # Mark untouched\n\n# Step 2: Split remaining into train (50%), validation (20%), and test (20%)\ndf_train, df_temp = train_test_split(df_train_val, test_size=0.40, random_state=42)\ndf_train = df_train.copy()\ndf_temp = df_temp.copy()\ndf_train["set_flag"] = 1  # Mark train\n\ndf_val, df_test = train_test_split(df_temp, test_size=0.50, random_state=42)\ndf_val = df_val.copy()\ndf_test = df_test.copy()\ndf_val["set_flag"] = 2  # Mark validation\ndf_test["set_flag"] = 3  # Mark test\n\n# Combine all subsets back into one dataframe\ndf_final = pd.concat([df_train, df_val, df_test, df_untouched], ignore_index=True)\n\n# Save as Parquet for efficient storage\

In [8]:
# Dimension of training set
df.shape

(2925493, 142)

# Data Processing

## 1. Drop features with > 50% missing values

In [9]:
## Drop features with > 50% missing values
missing_percent = (df.isna().sum() / len(df))* 100
cols_to_drop = missing_percent[missing_percent > 50].index
print(cols_to_drop)
df_dropped = df.drop(columns=cols_to_drop)

Index(['mths_since_last_delinq', 'mths_since_last_record', 'next_pymnt_d',
       'mths_since_last_major_derog', 'annual_inc_joint', 'dti_joint',
       'verification_status_joint', 'mths_since_recent_bc_dlq',
       'mths_since_recent_revol_delinq', 'revol_bal_joint',
       'sec_app_fico_range_low', 'sec_app_fico_range_high',
       'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths',
       'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_revol_util',
       'sec_app_open_act_il', 'sec_app_num_rev_accts',
       'sec_app_chargeoff_within_12_mths',
       'sec_app_collections_12_mths_ex_med', 'hardship_type',
       'hardship_reason', 'hardship_status', 'deferral_term',
       'hardship_amount', 'hardship_start_date', 'hardship_end_date',
       'payment_plan_start_date', 'hardship_length', 'hardship_dpd',
       'hardship_loan_status', 'orig_projected_additional_accrued_interest',
       'hardship_payoff_balance_amount', 'hardship_last_payment_amount'],
      dtype='object')


In [10]:
# Identify columns with string (object) content
string_columns = df_dropped.select_dtypes(include=['object']).columns.tolist()

# Display the list of column names containing string content
string_columns

['id',
 'term',
 'int_rate',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'earliest_cr_line',
 'revol_util',
 'initial_list_status',
 'last_pymnt_d',
 'last_credit_pull_d',
 'application_type',
 'hardship_flag',
 'debt_settlement_flag']

# Categorize Predictor Variable to Loss, Good and Other

In [11]:
# create categories for loan status
loss = ['Charged Off', 'Does not meet the credit policy. Status:Charged Off', 'Default']
good = ['Fully Paid', 'Current', 'Does not meet the credit policy. Status:Charged Off', 'Issued']
df_dropped['loan_category'] = df_dropped['loan_status'].apply(lambda x: 'Loss' if x in loss else ('Good' if x in good else 'Other'))

In [12]:
# filter only good and loss
df_dropped = df_dropped[df_dropped['loan_category'].isin(['Loss', 'Good'])]
# convert to numerical encoding
df_dropped['loan_cat_numerical'] = df_dropped['loan_category'].map({'Loss': 0, 'Good': 1})
df_dropped['loan_cat_numerical'].value_counts()

loan_cat_numerical
1    2530861
0     363742
Name: count, dtype: int64

# Add description column with existing borrower information from dataset

In [13]:
def generate_description(row):
    description = ""

    # Loan purpose and amount
    if pd.notna(row['purpose']):
        purpose_clean = row['purpose'].replace('_', ' ')
        description += f"I am applying for a ${int(row['loan_amnt']):,} loan to cover {purpose_clean}."
    else:
        description += f"I am applying for a ${int(row['loan_amnt']):,} loan."

    # Add title if it's different from purpose
    if pd.notna(row['title']) and pd.notna(row['purpose']):
        title_clean = str(row['title']).strip().lower()
        if title_clean != row['purpose'].replace('_', ' ').lower():
            description += f" More specifically, it is for {title_clean}."
    elif pd.notna(row['title']):
        title_clean = str(row['title']).strip().lower()
        description += f" More specifically, it is for {title_clean}."

    # Employment title and length
    emp_title = row['emp_title']
    emp_length = row['emp_length']

    if pd.notna(emp_title):
        description += f" I work at {emp_title.lower()}"
        if pd.notna(emp_length):
            emp_length_clean = emp_length.strip()
            if emp_length_clean == "10+ years":
                emp_length_clean = "over 10 years"
            elif emp_length_clean == "< 1 year":
                emp_length_clean = "less than 1 year"
            description += f" with {emp_length_clean} of experience"
        description += "."
    elif pd.notna(emp_length):
        emp_length_clean = emp_length.strip()
        if emp_length_clean == "10+ years":
            emp_length_clean = "over 10 years"
        elif emp_length_clean == "< 1 year":
            emp_length_clean = "less than 1 year"
        description += f" I have {emp_length_clean} of professional experience."

    # Income
    if pd.notna(row['annual_inc']):
        description += f" My annual income is approximately ${int(row['annual_inc']):,}."

    # Home ownership
    if pd.notna(row['home_ownership']):
        description += f" I currently {row['home_ownership'].lower()} my home."

    # Credit account info
    if pd.notna(row['open_acc']) and pd.notna(row['total_acc']):
        description += f" I have {int(row['open_acc'])} open credit accounts and {int(row['total_acc'])} total credit accounts."
    elif pd.notna(row['open_acc']):
        description += f" I have {int(row['open_acc'])} open credit accounts."
    elif pd.notna(row['total_acc']):
        description += f" I have {int(row['total_acc'])} total credit accounts."

    # Location
    if pd.notna(row['addr_state']):
        description += f" I am based in {row['addr_state']}."

    return description

In [14]:
df_dropped['description'] = df_dropped.apply(generate_description, axis=1)

In [15]:
df_dropped.head()

,Unnamed: 0,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,loan_category,loan_cat_numerical,description
0,0,1077501,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,735.0,739.0,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.0,0.0,5863.155187,5833.84,5000.00,863.16,0.00,0.0,0.00,Jan-2015,171.62,May-2020,704.0,700.0,0.0,1.0,Individual,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,N,N,Good,1,"I am applying for a $5,000 loan to cover credi..."
1,1,1077430,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,car,bike,309xx,GA,1.00,0.0,Apr-1999,740.0,744.0,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.0,0.0,1014.530000,1014.53,456.46,435.17,0.00,122.9,1.11,Apr-2013,119.66,Oct-2016,499.0,0.0,0.0,1.0,Individual,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,N,N,Loss,0,"I am applying for a $2,500 loan to cover car. ..."
2,2,1077175,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,735.0,739.0,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.0,0.0,3005.666844,3005.67,2400.00,605.67,0.00,0.0,0.00,Jun-2014,649.91,Jun-2017,739.0,735.0,0.0,1.0,Individual,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,N,N,Good,1,"I am applying for a $2,400 loan to cover small..."
3,3,1076863,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,other,personel,917xx,CA,20.00,0.0,Feb-1996,690.0,694.0,1.0,10.0,0.0,5598.0,21%,37.0,f,0.0,0.0,12231.890000,12231.89,10000.00,2214.92,16.97,0.0,0.00,Jan-2015,357.48,Apr-2016,604.0,600.0,0.0,1.0,Individual,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN

In [16]:
df_dropped['description'].iloc[17650]

'I am applying for a $3,000 loan to cover major purchase. More specifically, it is for personal pool. I work at east baton rouge school with 3 years of experience. My annual income is approximately $45,000. I currently mortgage my home. I have 25 open credit accounts and 36 total credit accounts. I am based in LA.'

# Risk Scoring using FinBERT

In [18]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

# Load FinBERT
model_name = "yiyanghkust/finbert-tone"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Create pipeline
finbert = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [19]:
def get_risk_score(text):
    result = finbert(text[:512])[0]  # Limit to 512 tokens max for BERT
    label = result['label'].lower()

    if label == 'positive':
        return 1
    elif label == 'neutral':
        return 2
    elif label == 'negative':
        return 3
    else:
        return None  # fallback

In [20]:
# Apply risk scoring to df_dropped['description'] subset for testing
df_sample = df_dropped.head(200)
df_sample['risk_score'] = df_sample['description'].apply(get_risk_score)

/var/folders/1k/m19_s5vj13nd1vhzx_rcdr780000gn/T/ipykernel_69165/2077250823.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample['risk_score'] = df_sample['description'].apply(get_risk_score)


In [21]:
df_sample['risk_score']

0      2
1      2
2      2
3      2
4      2
5      2
6      2
7      2
8      2
9      2
10     2
11     2
12     2
13     2
14     2
15     2
16     2
17     2
18     2
19     2
20     2
21     2
22     2
23     2
24     2
25     2
26     2
27     2
28     2
29     2
30     2
31     2
32     2
33     2
34     2
35     2
36     2
37     2
38     2
39     2
40     2
41     2
42     2
43     2
44     2
45     2
46     2
47     2
48     2
49     2
50     2
51     2
52     2
53     2
54     2
55     2
56     2
57     2
58     2
59     2
60     2
61     2
62     2
63     2
64     2
65     2
66     2
67     2
68     2
69     2
70     2
71     2
72     2
73     2
74     2
75     2
76     2
77     2
78     2
79     2
80     2
81     2
82     2
83     2
84     2
85     2
86     2
87     2
88     2
89     2
90     2
91     2
92     2
93     2
94     2
95     2
96     2
97     2
98     2
99     2
100    2
101    2
102    2
103    2
104    2
105    2
106    2
107    2
108    2
109    2
110    2
1

# Risk Scoring using OpenAI's GPT-3.5 / GPT-4 (via API)

In [27]:
!pip install --upgrade openai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [29]:
import openai

client = openai.OpenAI(api_key="your-api")

def classify_risk_with_gpt(description):
    prompt = (
        f"Given the following loan application description, assess the risk level:\n\n"
        f"{description}\n\n"
        f"Respond with just one word: Low, Medium, or High."
    )

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a financial risk analysis assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3
        )

        reply = response.choices[0].message.content.strip().lower()

        if "low" in reply:
            return 1
        elif "medium" in reply:
            return 2
        elif "high" in reply:
            return 3
        else:
            return None

    except Exception as e:
        print("Error:", e)
        return None

In [32]:
# test it on one description
test_description = df_dropped['description'].iloc[0]
print("DESCRIPTION:\n", test_description)

score = classify_risk_with_gpt(test_description)
print("Risk Score:", score)

DESCRIPTION:
 I am applying for a $5,000 loan to cover credit card. More specifically, it is for computer. I have over 10 years of professional experience. My annual income is approximately $24,000. I currently rent my home. I have 3 open credit accounts and 9 total credit accounts. I am based in AZ.
Risk Score: 2


In [35]:
# apply it to small sample
sample_df = df_dropped.sample(10, random_state=42).copy()

pd.set_option('display.max_colwidth', None)
sample_df['risk_score'] = sample_df['description'].apply(classify_risk_with_gpt)

In [37]:
print(sample_df[['description', 'risk_score']])

                                                                                                                                                                                                                                                                                                                          description  \
2731369                                                                                  I am applying for a $1,375 loan to cover car. More specifically, it is for car financing. My annual income is approximately $19,000. I currently rent my home. I have 4 open credit accounts and 14 total credit accounts. I am based in NJ.   
1881592                                                                                                               I am applying for a $4,800 loan to cover debt consolidation. My annual income is approximately $15,396. I currently rent my home. I have 7 open credit accounts and 24 total credit accounts. I am based in NY.   
1349691      

# Convert String to Numeric

## 1. Convert id to numeric

In [ ]:
## convert id to numeric
df_dropped['id'] = pd.to_numeric(df_dropped['id'], errors='coerce').astype('Int64')
# check if there'are any NAs
df_dropped['id'].isna().any()
df_dropped['id'].isna().sum()

# show the column with NA id
df_dropped[df_dropped['id'].isna()]

# drop the entire row 39786 because it's empty
df_dropped.shape

In [ ]:
# check result
df_dropped.head()

## 2. Convert emp_title to Numeric (Drop)

Reasons to drop feature emp_title:

1. High cardinality -- 372,749 unique values in 1,579,764 rows (~24% unique), making it difficult to extract meaningful patterns.
2. Encoding challenge -- One-hot encoding is impractical due to excessive feature expansion; label encoding introduces arbitrary ordinal relationships

In [ ]:
## Drop emp_title column (too many distinct emp_titles)
df_dropped.drop(columns=["emp_title"], inplace=True)

df_dropped.head()

## 3. Convert home_ownership to Numerical (One-hot Encoding)

In [ ]:
## Convert home_ownership into numerical
df_dropped['home_ownership'].unique()

df_dropped['home_ownership'].isnull().sum()

# perform one-hot encoding 
df_dropped = pd.get_dummies(df_dropped, columns=['home_ownership'], drop_first=False)

# convert the true/false into 1/0
home_ownership_cols = [col for col in df_dropped.columns if col.startswith('home_ownership_')]
df_dropped[home_ownership_cols] = df_dropped[home_ownership_cols].astype(int)
df_dropped.head()

## 4. Convert verification_status to Numerical (One-hot Encoding)

In [ ]:
## convert verification_status
df_dropped['verification_status'].unique()
df_dropped['verification_status'].isnull().sum()

# one-hot encoding 
df_dropped = pd.get_dummies(df_dropped, columns=['verification_status'], drop_first=False)

# convert true/false to 1/0
verification_status_cols = [col for col in df_dropped.columns if col.startswith('verification_status_')]
df_dropped[verification_status_cols] = df_dropped[verification_status_cols].astype(int)
df_dropped.head()

## 5. Convert issue_d to Numerical 

In [ ]:
## convert issue_d
df_dropped['issue_d'].head()

df_dropped['issue_d'].isnull().sum()

df_dropped['loan_status'].unique()

# Convert issue_d to datetime
df_dropped['issue_d'] = pd.to_datetime(df_dropped['issue_d'], format='%b-%Y')
# Group by month-year and calculate the proportion of "loss" loans
loss_rate = df_dropped.groupby(df_dropped['issue_d'].dt.to_period('Y'))['loan_category'].apply(lambda x: (x == 'Loss').mean())

# Plot the trend
plt.figure(figsize=(14, 6))
#x_labels = loss_rate.index.astype(str)[::4]  # Show every 4th label
plt.plot(loss_rate.index.astype(str), loss_rate.values, marker='o', linestyle='-')
#plt.xticks(ticks=x_labels, rotation=45)
plt.xticks(rotation=45)
plt.xlabel("Loan Issue Date (Year)")
plt.ylabel("Proportion of Loss Loans")
plt.title("Trend of Loan Default Rate Over Time (Yearly)")
plt.grid(True)
plt.show()

In [ ]:
# convert issue_d to an Ordinal Feature
df_dropped['issue_d_ordinal'] = df_dropped['issue_d'].dt.year - df_dropped['issue_d'].dt.year.min()
df_dropped.drop(columns=['issue_d'], inplace=True)  # Drop original datetime column

## 6. Convert url to Numeric (Drop)
Reason to drop url: 
1. url unaccessible

In [ ]:
## loanDetails that requires investor account to login (Unaccessible)
## Drop column
df_dropped.drop(columns=["url"], inplace=True)

df_dropped.head()

## 7. Convert Term to Numeric

In [ ]:
# Check feature Term and convert to numeric
df_dropped["term"].unique()

# Convert  numeric 
df_dropped["term"] = df_dropped["term"].str.extract("(\d+)").astype(float)

# Plot distribution of loan terms
plt.figure(figsize=(6,4))
sns.countplot(x=df_dropped["term"])
plt.title("Distribution of Loan Terms")
plt.xlabel("Loan Term (Months)")
plt.ylabel("Count")
plt.xticks(rotation=0)
plt.show()

## 8. Convert int_rate to Numeric

In [ ]:
# Check feature int_rate and convert to numeric
df_dropped["int_rate"].unique()

# Plot distribution of int_rate
plt.figure(figsize=(6,4))
sns.countplot(x=df_dropped["int_rate"])
plt.title("Distribution of int_rate")
plt.xlabel("int_rate)")
plt.ylabel("Count")
plt.xticks(rotation=0)
plt.show()

# Convert
df_dropped["int_rate"] = df_dropped["int_rate"].replace("nan", np.nan)
df_dropped["int_rate"] = df_dropped["int_rate"].str.replace("%", "").astype(float) / 100
df_dropped["int_rate"].head()

## 9. Convert sub_grade to Numeric (Drop grade)

In [ ]:
# Check feature grade and subgrade
df_dropped["grade"].unique()
df_dropped["sub_grade"].unique()

# Possible feature engineering: Combine into one feature A=1, B=2, C=3, D=4, E=5, F=6, G=7 (Smaller number has lower risk)===> 
# Use only converted sub_grade, drop feature grade
df_dropped = df_dropped.drop(["grade"], axis=1)

# Define base values for grades (lower = better credit, higher = higher risk)
grade_mapping = {"A" :1, "B" : 2, "C" : 3, "D" : 4, "E" : 5, "F" : 6, "G" : 7}

# Convert nan to np.nan
df_dropped["sub_grade"] = df_dropped["sub_grade"].replace("nan", np.nan)

# Check if has nan valus (1 nan)
df_dropped["sub_grade"].isna().sum()

# Check if the original "nan" value converted to np.nan (All converted)
print((df_dropped["sub_grade"] == "nan").sum())

# Plot the distribution of sub_grade with proper ranking from low to high
plt.figure(figsize=(12, 6))
sns.countplot(y=df_dropped["sub_grade"], order=sorted(df_dropped["sub_grade"].unique()), palette="Blues_r")
plt.title("Distribution of Sub Grade (Ranked Low to High)")
plt.xlabel("Count")
plt.ylabel("Sub Grade")
plt.show()


# Convert sub_grade into an ordered numeric feature where A1 is lowest risk and G5 is highest risk
df_dropped["sub_grade"] = df_dropped["sub_grade"].apply(lambda x: grade_mapping[str(x)[0]] * 10 + int(str(x)[1]) if pd.notna(x) else np.nan)

## 10. Convert emp_length to Numeric

In [ ]:
# Check feature emp_length
df_dropped["emp_length"].unique()

def convert_emp_length(emp):
    if pd.isna(emp):  # Handle missing values
        return np.nan
    if emp == "10+ years":
        return 10
    elif emp == "< 1 year":
        return 0
    else:
        return int(emp.split()[0])  # Extract the number from "X years"
## Check below for convertion criterion


## Convert emp_length to numeric
df_dropped["emp_length"] = df_dropped["emp_length"].apply(convert_emp_length)

In [ ]:
# Handeling 'purpose', 'title', 'zip_code', 'addr_state',

df_dropped["purpose"].unique()

## 11. Convert Purpose to Numeric


In [ ]:
# Convert "purpose" from string to numeric
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
le = LabelEncoder()

# Fit and transform the 'purpose' column
df_dropped["purpose"] = le.fit_transform(df_dropped["purpose"])

# View unique mappings
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(label_mapping)

## 12.Convert Title to Numeric (Drop)

In [ ]:
# Convert 'title' from strings to numeric (Since similar to "Purpose, drop it")
num_unique_titles = df_dropped["title"].nunique()
title_counts = df_dropped["title"].value_counts()
purpose_counts = df_dropped["purpose"].value_counts()

df_dropped = df_dropped.drop(columns=["title"])

## 13. Convert zip_code, addr_state to Numeric(Drop)

In [ ]:
# Convert "zip_code" to numeric
df_dropped["zip_code"].unique()

# Drop zip_code
df_dropped = df_dropped.drop(columns=["zip_code"])

# Drop "addr_state"
df_dropped["addr_state"].unique()
df_dropped = df_dropped.drop(columns=["addr_state"])

## 14. Convert hardship_flag to Numeric (Categorical{0,1})

In [ ]:
# Map 'N' to 0 and 'Y' to 1
df_dropped["hardship_flag"] = df_dropped["hardship_flag"].map({
                                                                          "N": 0, "Y": 1})

df_dropped["hardship_flag"].unique()

## 15. Convert debt_settlement_flag to Numeric (Categorical{0,1})

In [ ]:
# Map 'N' to 0 and 'Y' to 1
df_dropped["debt_settlement_flag"] = df_dropped["debt_settlement_flag"].map({
                                                                                        "N": 0, "Y": 1})

df_dropped["debt_settlement_flag"].unique()

## 16. Convert application_type to Numeric

In [ ]:
# Map 'Individual' to 0 and 'Joint App' to 1
df_dropped["application_type"] = df_dropped["application_type"].map(
    {"Individual": 0, "Joint App": 1})

df_dropped["application_type"].unique()

## 17. Convert last_credit_pull_d to Numeric
Convert date to a numeric measure of recency in days (reference date - credit pull date; smaller is worse)

Reference Date: September 30, 2020 because the dataset appears to be as recent as Q3 2020.

In [ ]:
df_dropped["last_credit_pull_d"] = pd.to_datetime(
    df_dropped["last_credit_pull_d"])

# Define reference date as the end of Q3 2020
reference_date = pd.to_datetime("2020-09-30")

df_dropped["last_credit_pull_d"] = (
    reference_date - df_dropped["last_credit_pull_d"]).dt.days

df_dropped["last_credit_pull_d"].head()

## 18. Convert earliest_cr_line to Numeric
In credit scoring models (e.g., FICO score), one of the key factors is the length of credit history, which refers to how long the borrower has had a credit account. Lenders care about how long you have had credit not the exactly date.

In [ ]:
df_dropped["earliest_cr_line"].unique()

In [ ]:
from datetime import datetime
df_dropped["earliest_cr_line"] = pd.to_datetime(df_dropped["earliest_cr_line"], format="%b-%Y")
df_dropped["earliest_cr_line"] = (
    (datetime.today().year - df_dropped["earliest_cr_line"].dt.year) * 12 +
    (datetime.today().month - df_dropped["earliest_cr_line"].dt.month)
)

In [ ]:
df_dropped["earliest_cr_line"].unique()

## 19. Convert revol_util to Numeric

In [ ]:
df_dropped["revol_util"].unique()

In [ ]:
df_dropped["revol_util"] = pd.to_numeric(df_dropped["revol_util"].str.rstrip("%"), errors="coerce") / 100

## 20. Convert initial_list_status to Numeric

In [ ]:
# Map 'w' to 0 and 'f' to 1
df_dropped["initial_list_status"] = df_dropped["initial_list_status"].map({"w": 0, "f": 1})

In [ ]:
df_dropped["initial_list_status"].unique()

## 21. Convert last_pymnt_d to Numeric
Months Since Last Payment = (Current Year − Last Payment Year) × 12 + (Current Month − Last Payment Month)

In [ ]:
from datetime import datetime
import pandas as pd

# Ensure the column is in datetime format
df_dropped["last_pymnt_d"] = pd.to_datetime(df_dropped["last_pymnt_d"], errors='coerce')

# Calculate months since last payment
df_dropped["months_since_last_pymnt"] = (
    (datetime.today().year - df_dropped["last_pymnt_d"].dt.year) * 12 +
    (datetime.today().month - df_dropped["last_pymnt_d"].dt.month)
)

In [ ]:
df_dropped["months_since_last_pymnt"].unique()

In [ ]:
# Check df after column dropping
df_dropped.head()

In [ ]:
# Select only numeric features before computing correlation
df_numeric = df_dropped.select_dtypes(include=["number"])  # Keep only numeric columns

# Compute correlations with target
correlations = df_numeric.corr()["loan_cat_numerical"].dropna().sort_values(ascending=False)

In [ ]:
# Plot correlations
plt.figure(figsize=(12, 6))
correlations.plot(kind="bar")
plt.title("Feature Correlation with Target (loan_cat_numerical)")
plt.ylabel("Correlation Coefficient")
plt.grid()
plt.show()

# use the entire training dataset (df_train_dropped) for model training

In [ ]:
'''
import pickle
from sklearn.model_selection import train_test_split

# Step 1: Select exactly 60,000 rows from df_train_dropped
df_selected = df_train_dropped.sample(n=1000000, random_state=42)  # Ensures reproducibility

# Step 2: Split selected data into train (60%), validation (20%), and test (20%)
df_train, df_temp = train_test_split(df_selected, test_size=0.40, random_state=42)
df_val, df_test = train_test_split(df_temp, test_size=0.50, random_state=42)

# Print dataset distribution
print(f"Training Set: {len(df_train)} rows ({len(df_train)/len(df_selected)*100:.1f}%)")
print(f"Validation Set: {len(df_val)} rows ({len(df_val)/len(df_selected)*100:.1f}%)")
print(f"Test Set: {len(df_test)} rows ({len(df_test)/len(df_selected)*100:.1f}%)")
'''

In [ ]:
# Split Dataset Randomly
from sklearn.model_selection import train_test_split

# Step 1: Split off untouched data (10%)
df_train_val, df_untouched = train_test_split(df_dropped, test_size=0.10, random_state=42)
df_train_val = df_train_val.copy()  # Avoid SettingWithCopyWarning
df_untouched = df_untouched.copy()
df_untouched["set_flag"] = 0  # Mark untouched

# Step 2: Split remaining into train (50%), validation (20%), and test (20%)
df_train, df_temp = train_test_split(df_train_val, test_size=0.40, random_state=42)
df_train = df_train.copy()
df_temp = df_temp.copy()
df_train["set_flag"] = 1  # Mark train

df_val, df_test = train_test_split(df_temp, test_size=0.50, random_state=42)
df_val = df_val.copy()
df_test = df_test.copy()
df_val["set_flag"] = 2  # Mark validation
df_test["set_flag"] = 3  # Mark test

# Combine all subsets back into one dataframe
df_final = pd.concat([df_train, df_val, df_test, df_untouched], ignore_index=True)

# Save as Parquet for efficient storage
df_final.to_parquet("dataset_with_flags.parquet", index=False)

# Check dataset sizes
print(f"Training Set: {len(df_train)} rows ({len(df_train)/len(df)*100:.1f}%)")
print(f"Validation Set: {len(df_val)} rows ({len(df_val)/len(df)*100:.1f}%)")
print(f"Test Set: {len(df_test)} rows ({len(df_test)/len(df)*100:.1f}%)")
print(f"Untouched Set: {len(df_untouched)} rows ({len(df_untouched)/len(df)*100:.1f}%)")

print("Data saved with set_flag column.")

In [ ]:
# Seperate into pre-loan and post-loan features
# Creating Python lists for categorized features

pre_loan_features_list = [
    "acc_open_past_24mths", "addr_state", "all_util", "annual_inc", "annual_inc_joint",
    "application_type", "avg_cur_bal", "bc_open_to_buy", "bc_util", "dti", "dti_joint",
    "delinq_2yrs", "emp_length", "emp_title", "fico_range_high", "fico_range_low",
    "grade", "home_ownership", "il_util", "inq_fi", "inq_last_12m", "inq_last_6mths",
    "installment", "int_rate", "issue_d", "loan_amnt", "loan_status", "max_bal_bc",
    "mo_sin_old_il_acct", "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op",
    "mo_sin_rcnt_tl", "mort_acc", "mths_since_last_delinq", "mths_since_last_major_derog",
    "mths_since_last_record", "mths_since_rcnt_il", "mths_since_recent_bc",
    "mths_since_recent_bc_dlq", "mths_since_recent_inq", "mths_since_recent_revol_delinq",
    "num_accts_ever_120_pd", "num_actv_bc_tl", "num_actv_rev_tl", "num_bc_sats",
    "num_bc_tl", "num_il_tl", "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "open_acc", "open_acc_6m", "orig_projected_additional_accrued_interest",
    "pct_tl_nvr_dlq", "verification_status", "total_acc", "verified_status_joint",
    "term", "total_bal_ex_mort", "pub_rec_bankruptcies", "debt_settlement_flag",
    "tot_cur_bal", "open_act_il", "sec_app_open_acc", "tot_coll_amt", "tax_liens",
    "disbursement_method", "percent_bc_gt_75", "sec_app_num_rev_accts",
    "sec_app_revol_util", "revol_bal_joint", "total_il_high_credit_limit",
    "sec_app_fico_range_low", "sub_grade", "sec_app_mths_since_last_major_derog",
    "sec_app_chargeoff_within_12_mths", "total_bc_limit", "tot_hi_cred_lim",
    "pymnt_plan", "sec_app_fico_range_high", "sec_app_earliest_cr_line",
    "total_bal_il", "purpose", "num_tl_op_past_12m", "member_id", "title",
    "zip_code", "sec_app_open_act_il", "sec_app_collections_12_mths_ex_med",
    "id", "total_rev_hi_lim", "earliest_cr_line", "initial_list_status",
    "total_cu_tl", "open_rv_12m", "pub_rec", "sec_app_inq_last_6mths",
    "funded_amnt", "open_il_12m", "sec_app_mort_acc"
]

post_loan_features_list = [
    "acc_now_delinq", "chargeoff_within_12_mths", "collection_recovery_fee",
    "collections_12_mths_ex_med", "delinq_amnt", "last_credit_pull_d",
    "last_fico_range_high", "last_fico_range_low", "last_pymnt_amnt",
    "last_pymnt_d", "next_pymnt_d", "num_tl_120dpd_2m", "num_tl_30dpd",
    "num_tl_90g_dpd_24m", "out_prncp", "out_prncp_inv", "policy_code",
    "recoveries", "revol_bal", "revol_util", "total_pymnt", "total_pymnt_inv",
    "total_rec_int", "total_rec_prncp", "settlement_percentage",
    "hardship_reason", "desc", "hardship_start_date", "settlement_term",
    "hardship_end_date", "hardship_length", "hardship_last_payment_amount",
    "hardship_dpd", "hardship_payoff_balance_amount", "hardship_amount",
    "deferral_term", "hardship_loan_status", "hardship_flag", "settlement_date",
    "settlement_amount", "hardship_status", "debt_settlement_flag_date",
    "payment_plan_start_date", "open_rv_24m", "settlement_status",
    "total_rec_late_fee", "hardship_type", "url"
]




In [ ]:
## Filter pre-loan features
drop_list = list(cols_to_drop)  # Convert Index to list
drop_list.extend(["emp_title", "url", "grade", "title", "zip_code", "addr_state",'verified_status_joint', 'disbursement_method', 'sec_app_mths_since_last_major_derog', 'member_id',"loan_status", "issue_d", "pymnt_plan"])

filter_pre_loan_features = [feature for feature in pre_loan_features_list if feature not in drop_list]

# Ensure only existing features are used
filter_pre_loan_features = [feature for feature in filter_pre_loan_features if feature in df_train.columns]
# Remove leaky variables
leakage_features = ["grade", "sub_grade", "int_rate", "loan_status", "id", "member_id", "issue_d", "funded_amnt", "initial_list_status", "term", "debt_settlement_flag",
                    "num_tl_30dpd", "num_tl_120dpd_2m", "num_tl_90g_dpd_24m", "num_tl_op_past_12m"]
filter_pre_loan_features = [feature for feature in filter_pre_loan_features if feature not in leakage_features]

X_train, y_train = df_train[filter_pre_loan_features], df_train.get("loan_cat_numerical")
X_val, y_val = df_val[filter_pre_loan_features], df_val.get("loan_cat_numerical")
X_test, y_test = df_test[filter_pre_loan_features], df_test.get("loan_cat_numerical")


# Save datasets using pickle
with open("train_val_test_2.pkl", "wb") as f:
    pickle.dump((X_train, X_val, X_test, y_train, y_val, y_test), f)

print("Data split and saved as 'train_val_test_2.pkl' for training in another notebook.")

In [ ]:
'''
# Select features and target from the existing train-validation-test split
X_train, y_train = df_dropped[all_features], df_dropped["loan_cat_numerical"]
X_val, y_val = df_val[all_features], df_val["loan_cat_numerical"]
X_test, y_test = df_test[all_features], df_test["loan_cat_numerical"]

# Save datasets using pickle
with open("train_val_test_complete.pkl", "wb") as f:
    pickle.dump((X_train, X_val, X_test, y_train, y_val, y_test), f)

print("Data split and saved as 'train_val_test_complete.pkl' for training in another notebook.")
'''

In [ ]:
#len(filter_pre_loan_features)

In [ ]:
# Define pre-loan features and target
'''
selected_features = [
    "annual_inc", "acc_open_past_24mths", "all_util",
    "delinq_2yrs", "dti", "emp_length", "inq_last_6mths",
    "loan_amnt", "open_acc", "pub_rec", "purpose", "revol_bal",
    "term", "home_ownership_ANY", "home_ownership_MORTGAGE", "home_ownership_NONE",
    "home_ownership_OTHER", "home_ownership_OWN", "home_ownership_RENT"
]

# Ensure only existing features are used
filter_pre_loan_features = [feature for feature in filter_pre_loan_features if feature in df_train.columns]

X_train, y_train = df_train[filter_pre_loan_features], df_train.get("loan_cat_numerical")
X_val, y_val = df_val[filter_pre_loan_features], df_val.get("loan_cat_numerical")
X_test, y_test = df_test[filter_pre_loan_features], df_test.get("loan_cat_numerical")


# Save datasets using pickle
with open("train_val_test_1.pkl", "wb") as f:
    pickle.dump((X_train, X_val, X_test, y_train, y_val, y_test), f)

print("Data split and saved as 'train_val_test_1.pkl' for training in another notebook.")
'''